In [51]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [52]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
from glasses.nn.models.classification.mobilenet import MobileNetBasicBlock,MobileNetEncoder
from glasses.nn.models.classification.resnet import ResNetEncoder


MobileNetBasicBlock(16, 24)

MobileNetBasicBlock(
  (block): Residual(
    (block): Sequential(
      (0): ConvBnAct(
        (conv): Conv2dPad(16, 96, kernel_size=(1, 1), stride=(1, 1))
        (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU6()
      )
      (1): ConvBnAct(
        (conv): DeepthWiseConv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96)
        (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU6()
      )
      (2): Conv2dPad(96, 24, kernel_size=(1, 1), stride=(1, 1))
      (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)

In [115]:
from torchsummary import summary

encoder = MobileNetEncoder(
#               depths=[1, 2, 1, 1, 1, 1, 1],
              block=MobileNetBasicBlock)
x = torch.rand(1, 3, 224, 224)
model = encoder.cuda()
summary(model, (3, 224, 224))

# encoder(x).shape
# encoder

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 32, 112, 112]             896
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
  DeepthWiseConv2d-4         [-1, 32, 112, 112]             320
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
         Conv2dPad-7         [-1, 16, 112, 112]             528
         Conv2dPad-8         [-1, 96, 112, 112]           1,632
       BatchNorm2d-9         [-1, 96, 112, 112]             192
            ReLU6-10         [-1, 96, 112, 112]               0
 DeepthWiseConv2d-11           [-1, 96, 56, 56]             960
      BatchNorm2d-12           [-1, 96, 56, 56]             192
            ReLU6-13           [-1, 96, 56, 56]               0
        Conv2dPad-14           [-1, 24,

In [117]:
class DeepthWiseConv2d(nn.Conv2d):
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, groups=out_channels, **kwargs)

In [2]:
from torchvision.models import mobilenet_v2
from torchsummary import summary


model = mobilenet_v2().cuda()

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

### Residual Add Performance

In [24]:
class Residual(nn.Module):

    def __init__(self, block: nn.Module,
                 res_func: Callable[[Tensor], Tensor] = None,
                 shortcut: nn.Module = None, *args, **kwargs):
        super().__init__()
        self.block = block
        self.shortcut = shortcut
        self.res_func = res_func

    def forward(self, x: Tensor) -> Tensor:
        res = x
        x = self.block(x)
        if self.shortcut is not None:
            res = self.shortcut(res)
        if self.res_func is not None:
            x = self.res_func(x, res)
        return x


def add(x: Tensor, res: Tensor) -> Tensor:
    return x.add_(res)


ResidualAdd = partial(Residual, res_func=add)

In [25]:
%%timeit
m = ResidualAdd(nn.Sequential(
                    nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False),
                    nn.BatchNorm2d(64),
                    nn.ReLU(),
                    nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False),
                    nn.BatchNorm2d(64),
                
            ))

x = torch.rand(1,64, 48, 48)

for _ in range(10):
    m(x)

70.3 ms ± 6.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Subclass


In [26]:
class ResidualAdd(nn.Module):
    def __init__(self, block: nn.Module,
                 shortcut: nn.Module = None, *args, **kwargs):
        super().__init__()
        self.block = block
        self.shortcut = shortcut

    def forward(self, x: Tensor) -> Tensor:
        res = x
        x = self.block(x)
        if self.shortcut is not None:
            res = self.shortcut(res)
        x += res
        return x

In [27]:
%%timeit
m = ResidualAdd(nn.Sequential(
                    nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False),
                    nn.BatchNorm2d(64),
                    nn.ReLU(),
                    nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False),
                    nn.BatchNorm2d(64),
                
            ))

x = torch.rand(1,64, 48, 48)

for _ in range(10):
    m(x)

64.7 ms ± 2.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
from glasses.utils.PretrainedWeightsProvider import PretrainedWeightsProvider

# test if the weights are correct, only for resnet18 
PretrainedWeightsProvider()['resnet18']

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (block): Sequential(
          (0): ResNetBasicBlock(
            (block): Residual(
              (block): Sequential(
                (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (act1): ReLU(inplace=True)
                (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [10]:
from glasses.nn.models.classification.resnet import ResNetBasicBlock


ResNetBasicBlock(32, 64)

TypeError: __init__() missing 1 required positional argument: 'block'

In [11]:
from glasses.nn.models.classification import ResNet


ResNet.resnet18()

[autoreload of glasses.nn.models.classification.resnet failed: Traceback (most recent call last):
  File "/Users/vaevictis/anaconda3/envs/dl/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/vaevictis/anaconda3/envs/dl/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/vaevictis/anaconda3/envs/dl/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/vaevictis/anaconda3/envs/dl/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/vaevictis/anaconda3/envs/dl/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/vaevictis/anaconda3/envs/dl/lib/python3.7/site-packages/IPython/e

TypeError: __init__() missing 1 required positional argument: 'block'

In [4]:
from glasses.nn.models.classification.resnet import ResNetBasicBlock

x = torch.rand(1, 32, 48, 48)
block = ResNetBasicBlock(32, 64)
print(block)
block(x)

ResNetBasicBlock(
  (block): Residual(
    (block): Sequential(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (shortcut): ResNetShorcut(
      (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act): ReLU(inplace=True)
)


tensor([[[[0.3507, 0.0000, 0.2612,  ..., 0.0000, 0.0000, 1.0867],
          [0.0000, 0.9422, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.2224, 0.2569],
          ...,
          [0.4459, 0.3775, 3.1123,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 1.2234,  ..., 1.2519, 0.0000, 0.7491],
          [0.0000, 0.0000, 0.3720,  ..., 0.0000, 0.8578, 0.0000]],

         [[0.4809, 0.6557, 2.3329,  ..., 0.0000, 0.0000, 0.7809],
          [0.0000, 0.8320, 0.0000,  ..., 0.1316, 1.9552, 0.2474],
          [0.0000, 0.7629, 0.2113,  ..., 0.0000, 0.0000, 0.3037],
          ...,
          [0.0000, 0.0000, 2.3773,  ..., 0.0000, 0.0000, 0.6268],
          [0.0000, 0.0000, 0.8727,  ..., 1.7412, 0.0000, 0.2877],
          [0.0000, 2.0063, 3.6411,  ..., 2.3751, 0.0000, 1.7510]],

         [[0.6819, 0.0000, 0.6205,  ..., 0.0000, 0.9400, 1.5356],
          [0.0000, 1.3500, 1.0352,  ..., 0.8879, 2.3182, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0